# Stimulus playground
This script is a playground to inspect and manipulate stimuli for a binocular rivalry (BR) experiment<br>
In this experiment (as at April th) I aim to present objects that are associated with a color (e.g., banana - yellow, strawberry - red) in rivarous situations.<br>
1. Show the same stimulus either in the "correct" and in an "incorrect" color (a banana in yellow and blue)
2. Show two different stimuli that are associated with different colors but in the same color (a banana and a strawberry both in yellow)

Compare the onset and sustained dominance with rivaling gratings that follow a series of non-rivaling gratings that appear to be rotating (Denison et al., 2011, Attarha et al., 2015)

## Stimuli
In the following I will look at the stimuli used by Bannert and Bartels 2013

In [1]:
# import libraries

# use the os
import os
import glob
# math and data structure
import numpy as np
import pandas as pd
# plotting
import matplotlib.pyplot as plt
# image processing
import cv2

In [2]:
# set directories
stim_dir = os.path.join('..', 'fuer_vincent', 'stimuli', 'equalizedStimuli')
# get one example stimulus
banana = os.path.join(stim_dir, 'banana', 'banana.jpg')

In [3]:
# stimuli from Teichmann et al., 2020
# provided by her OSF site (https://osf.io/tcqjh/)
stim_dir = os.path.join('..','stimuli')
original_stim_dir = os.path.join(stim_dir,'original')
true_color_stim_dir = os.path.join(stim_dir,'true_color')
inverted_color_stim_dir = os.path.join(stim_dir,'inverted')
grey_stim_dir = os.path.join(stim_dir,'grey') # not used yet
# get a list of all stimuli in the directory
stimuli_full_path = glob.glob(os.path.join(original_stim_dir,'*.png'))
stimuli_full_path.sort()

# extract the image name from the stimulus path
stimuli = [os.path.basename(stim) for stim in stimuli_full_path]

In [225]:
# read out the original stimuli and remove the background
for stim_path, stim in zip(stimuli_full_path, stimuli):
    # output directories
    true_color_output_dir = os.path.join(true_color_stim_dir, stim)
    inverted_color_output_dir = os.path.join(inverted_color_stim_dir, stim)
    
    # read in the original image
    img = cv2.imread(stim_path)
    
    # convert image from BGR to HLS space
    hls_img = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    # create a copy of the hls_img and invert the hue
    inverted_hls_img = np.copy(hls_img)
    # select all pixels that are not white and increase the hue value by 90 (hue ranges from 0-179)
    inverted_hls_img[np.where(hls_img[:,:,1]!=255)]+=np.array([90,0,0],dtype='uint8')
    # convert inverted hls image back into BGR 
    inverted_bgr_img = cv2.cvtColor(inverted_hls_img, cv2.COLOR_HLS2BGR)
    
    # add alpha channel to original image by converting from BGR to BGRA space
    bgra_img = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)
    # add alpha channel to inverted image by converting from BGR to BGRA space
    inverted_bgra_img = cv2.cvtColor(inverted_bgr_img, cv2.COLOR_BGR2BGRA)
    
    # change alpha channel of all pixels that are white to completely transparent
    # CAUTION this results in pixels that are part of the stimulus to be transparent. 
    # Maybe I need to also create a mask based on the borders fot the stimulus
    bgra_img[np.where(hls_img[:,:,1]==255)] -= np.array([0,0,0,255],dtype='uint8') 
    inverted_bgra_img[np.where(hls_img[:,:,1]==255)] -= np.array([0,0,0,255],dtype='uint8')
    cv2.imwrite(true_color_output_dir,bgra_img)
    cv2.imwrite(inverted_color_output_dir, inverted_bgra_img)